In [1]:
!nvidia-smi

Sun Nov  8 19:50:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install keras-tcn

     |████████████████████████████████| 378kB 23.5MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from tcn import TCN, tcn_full_summary

In [5]:
os.chdir('gdrive/My Drive/Weave/Earables Project/Final Experiments/')

In [19]:
data = pickle.load(open('data/processed_df.pkl', 'rb'))

In [20]:
labels = data.iloc[:,-3:-2]
ohe = OneHotEncoder()
ohe.fit(labels)
n_classes = 32

In [21]:
rem_col = ['Gx_Right', 'Gy_Right', 'Gz_Right', 'Ax_Right', 'Ay_Right', 'Az_Right']

In [22]:
data = data.drop(columns=rem_col)

In [24]:
data.columns

Index(['Gx_Left', 'Gy_Left', 'Gz_Left', 'Ax_Left', 'Ay_Left', 'Az_Left',
       'label', 'subject', 'session'],
      dtype='object')

In [30]:
subjects = list(np.unique(data.subject))
# print(subjects)
test_i = 1
test_sub = subjects[test_i]

sub_data = data[data.subject == test_sub]
idx = list(sub_data.index)
indices = np.arange(idx[0][0], idx[-1][0]+1)
np.random.shuffle(indices)

k=5
folds = []
fold_len = len(indices)//k
for i in range(k):
    st = i * fold_len
    en = (i+1) * fold_len 
    fold_i = indices[st:en]
    fold_d = sub_data.loc[fold_i]
    folds.append(fold_d)

In [33]:
subjects = list(np.unique(data.subject))
# print(subjects)
for test_i in range(1,9):
    # test_i = 1
    test_sub = subjects[test_i]

    sub_data = data[data.subject == test_sub]
    idx = list(sub_data.index)
    indices = np.arange(idx[0][0], idx[-1][0]+1)
    np.random.shuffle(indices)

    k=5
    folds = []
    fold_len = len(indices)//k
    for i in range(k):
        st = i * fold_len
        en = (i+1) * fold_len 
        fold_i = indices[st:en]
        fold_d = sub_data.loc[fold_i]
        folds.append(fold_d)

    for test_fold in range(3):
        test_data = folds[test_fold]
        train_data = []
        for fold_i, fold_d in enumerate(folds):
            if fold_i != test_fold:
                train_data.append(fold_d)

        train_data = pd.concat(train_data)

        X_train = train_data.iloc[:,:-3].to_numpy()
        y_train = train_data.iloc[:,-3:-2]
        X_test = test_data.iloc[:,:-3].to_numpy()
        y_test = test_data.iloc[:,-3:-2]
        y_train = ohe.transform(y_train).toarray()
        y_test = ohe.transform(y_test).toarray()

        #--------------------------------------------
        tmp = []
        for i in range(len(X_train)):
            ch = []
            for j in range(6):
                ch.append(X_train[i][j])
            ch = np.array(ch).T
            tmp.append(ch)
        X_train = np.array(tmp)

        tmp = []
        for i in range(len(X_test)):
            ch = []
            for j in range(6):
                ch.append(X_test[i][j])
            ch = np.array(ch).T
            tmp.append(ch)
        X_test = np.array(tmp)
        del tmp, ch
        # print(X_train.shape, y_train.shape)
        # print(X_test.shape, y_test.shape)

        # --------------------------------------
        batch_size = 32
        input_shape = (100, 6)

        img_input = Input(shape=input_shape)
        x = TCN(return_sequences=False, nb_filters=64)(img_input)
        # x = TCN(return_sequences=False, nb_filters=128)(x)
        x = Dense(n_classes, activation='sigmoid', name='output')(x)

        model = Model(img_input, x, name='tcn_left_sub_{}_fold_{}'.format(test_sub, test_fold+1))
        # model.summary()

        # -----------------------------------------------
        model_name = 'tcn_left_sub_{}_fold_{}'.format(test_sub, test_fold+1)
        optimizer = Adam(learning_rate=0.001)
        # loss = BinaryCrossentropy()
        # model.load_weights('checkpoint/{}.h5'.format(model_name))
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

        checkpoint = ModelCheckpoint('checkpoint/left_vs_right/{}.h5'.format(model_name), monitor='val_loss', verbose=0, mode='min',save_best_only=True)
        earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
        reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, min_lr=0.000001)

        # ----------------------------------------------
        print('started training...')
        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            batch_size=batch_size,
                            epochs=100,
                            # workers=4,
                            callbacks=[checkpoint, earlystopping],
                            shuffle=True,
                            verbose=0)

        # -----------------------------------------------
        labels = ohe.categories_[0]
        model.load_weights('checkpoint/left_vs_right/{}.h5'.format(model_name))
        print('Model loaded: checkpoint/left_vs_right/{}.h5'.format(model_name))
        pred = model.predict(X_test)
        y_pred = [labels[i] for i in np.argmax(pred, axis=1)]
        y_true = [labels[i] for i in np.argmax(y_test, axis=1)]
        print('---------------------------------')
        print('sub: {}, fold: {} - acc {}'.format(test_sub, test_fold+1, accuracy_score(y_true, y_pred)))
        # print('---------------------------------')

started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_12_fold_1.h5
---------------------------------
sub: 12, fold: 1 - acc 0.6419354838709678
started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_12_fold_2.h5
---------------------------------
sub: 12, fold: 2 - acc 0.7105990783410139
started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_12_fold_3.h5
---------------------------------
sub: 12, fold: 3 - acc 0.7127708621484555
started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_13_fold_1.h5
---------------------------------
sub: 13, fold: 1 - acc 0.9745098039215686
started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_13_fold_2.h5
---------------------------------
sub: 13, fold: 2 - acc 0.9808823529411764
started training...
Model loaded: checkpoint/left_vs_right/tcn_left_sub_13_fold_3.h5
---------------------------------
sub: 13, fold: 3 - acc 0.9656862745098039
started training...
Model loaded: 